In [1]:
from typing import TypedDict, Annotated, List, Literal, Sequence, Optional, Dict
import operator
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_core.messages import BaseMessage

from langgraph.graph import MessagesState, StateGraph, START, END
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder, PromptTemplate, BasePromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.tools import tool
from langgraph.types import Command, interrupt
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.graph.message import add_messages
from langsmith import traceable

from langchain.agents import AgentExecutor
from langchain.agents import create_tool_calling_agent
from langchain.tools import Tool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.utilities import OpenWeatherMapAPIWrapper
from langchain_google_genai import ChatGoogleGenerativeAI

from dotenv import load_dotenv
import os

# Chatbot

In [2]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [3]:
api_key = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = api_key

def initialize_llm():
    """Initialize and return the LLM client"""
    return ChatGoogleGenerativeAI(
            model="gemini-2.0-flash",
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
)

llm = initialize_llm()

In [4]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

search = GoogleSerperAPIWrapper()
search_tool = Tool(
    name="google_search",
    func=search.run,
    description="Use this tool to search Google using Serper API.",
    return_direct=True
)

tools = [search_tool]
llm_with_tools = llm.bind_tools(tools)

In [17]:
memory = MemorySaver()
graph_builder = StateGraph(State)
graph_builder.add_node("chatbot", chatbot)

tool_node = ToolNode(tools=[search_tool])
graph_builder.add_node("tools", tool_node)
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")

graph_builder.set_entry_point("chatbot")
graph_builder.set_finish_point("chatbot")

graph = graph_builder.compile(
    checkpointer=memory,
    interrupt_after=["tools"],
)

config = {"configurable": {"thread_id": "1"}}

while True:
    user_input = input("User:")
    if user_input.lower() in ["q", "quit", "exit"]:
        print("GoodBye !!")
        break
    for event in graph.stream(
        {"messages": ("user", user_input)}, config, stream_mode="values"
    ):
        event["messages"][-1].pretty_print()

================================ Human Message =================================

Hi there
================================== Ai Message ==================================

Hello! How can I help you today?
================================ Human Message =================================

nothing
================================== Ai Message ==================================

Okay, have a great day! Let me know if you need anything in the future.
GoodBye !!
